# Getting Started with Miyagi's Retrieval Augmented Generation (RaG) Workflow using Azure Cognitive Search (ACS) and Semantic Kernel

To quickly get started, follow these steps:

1. Ensure the [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) is installed.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the `rag` folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.

> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

## Overview
In this notebook, we will show you how to quickly get started with Miyagi's Retrieval Augmented Generation (RaG) workflow using Semantic Kernel, Azure Cognitive Search (ACS) and Cosmos. Through this use case, involving stock recommendations, you will understand the basics of RaG and the capabilities of Semantic Kernel, Azure Cognitive Search and Cosmos to adapt to your specific use case.

## Load settings from .env file

In [ ]:
#r "nuget: dotenv.net"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

## Prepare the kernel with memory using Azure Cognitive Search

We first add the nuget packages (dependecies) required to instantiate the kernel. 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta6"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta6"

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;

With dependencies installed and configured, we create an instance of the kernel using the built-in `Kernel.Builder` class

In [ ]:
var kernel = Kernel.Builder

    .WithAzureOpenAIChatCompletionService(
        env["AZURE_OPENAI_CHAT_MODEL"],
        env["AZURE_OPENAI_ENDPOINT"],
        env["AZURE_OPENAI_API_KEY"]
    )
        
    .Build();

To make the kernel stateful, we will use Memory Plugin with a Azure Cognitive Search and Embedding backend. 

In [ ]:
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch;
var memoryBuilder = new MemoryBuilder();
memoryBuilder
    .WithAzureOpenAITextEmbeddingGenerationService(
        env["AZURE_OPENAI_EMBEDDING_MODEL"],
        env["AZURE_OPENAI_ENDPOINT"],
        env["AZURE_OPENAI_API_KEY"]
    )

    .WithMemoryStore(
        new AzureCognitiveSearchMemoryStore(
            env["AZURE_COGNITIVE_SEARCH_ENDPOINT"],
            env["AZURE_COGNITIVE_SEARCH_API_KEY"]
        ));

var memory = memoryBuilder.Build();

## Vectorize and persist embeddings in Azure Cognitive Search with Semantic Kernel

Embeddings are a type of data transformation that convert text into high-dimensional vectors that capture semantic relationships between words or text chunks. This vectorized form allows machines to process text in a manner akin to numerical data, enabling operations like similarity comparison which is foundational for vector databases to retrieve meaningful context for LLMs. Embeddings are essential to generate and retrieve relevant content in a RaG workflow because:

1. They normalize your data in a common mathematical representation, making it machine-friendly
1. They compress data, retaining essential representations while reducing dimensionality
1. They preserve relationships within your data
1. They are representationally dense, meaning they are more efficient to store and process


> See a visual representation of the embeddings in 2D space below (from Cohere):

![Semantic Search](https://txt.cohere.com/content/images/2023/01/Vis-7-2.jpg)

Now, let’s delve into the process of creating and utilizing embeddings in the context of ACS.

In this scenario, we have a snippet of proprietary text data—extracted from a book—which needs to be processed and indexed for semantic search. The steps involve:

1. Chunking the Text: Breaking down the text into manageable pieces, referred to as 'chunks'.
1. Generating Embeddings: Transforming these chunks into high-dimensional vectors using an Azure Text Embedding Generation Service.
1. Persisting Embeddings to ACS: Storing these embeddings in Azure Cognitive Search to facilitate semantic search.

The provided code snippet orchestrates these steps. It reads the text data from a local file, chunks it, generates embeddings for each chunk, and then persists these embeddings to ACS. The specific operations are abstracted for simplicity using methods like SaveInformationAsync, which is part of a hypothetical kernel object in this code. The end result is a collection of indexed embeddings in ACS that significantly enhance search quality by moving beyond mere keyword matching to understanding semantic relationships.

In [ ]:
using Microsoft.SemanticKernel.Text;
using System.IO;

var dataset = "intelligent-investor.txt";
var recommendationServicePath = "../../../../services/recommendation-service/dotnet";
const int MaxTokensPerParagraph = 160;
const int MaxTokensPerLine = 60;

// Load text data from a local file
var filePath = Path.Combine(recommendationServicePath, "resources", "sample-datasets", dataset);
var streamReader = new StreamReader(filePath);
var text = await streamReader.ReadToEndAsync();

// Define a memory collection name for indexing
var memoryCollectionName = "userId";

// Chunk the text into lines and paragraphs for manageable processing
var lines = TextChunker.SplitPlainTextLines(text, MaxTokensPerLine);
var chunks = TextChunker.SplitPlainTextParagraphs(lines, MaxTokensPerParagraph);

// Loop through each chunk to generate embeddings and persist to ACS
for (var i = 0; i < chunks.Count; i++)
{
    var chunk = chunks[i];
    // Generate an identifier for each chunk
    var key = await memory.SaveInformationAsync(
        memoryCollectionName,
        chunk,
        $"{dataset}-{i}",
        $"Dataset: {dataset} Chunk: {i}",
        i.ToString());
}

// Output the result to console
System.Console.WriteLine($"Saved {chunks.Count} chunks to memory collection {memoryCollectionName}");


## Search and retrieve documents using Semantic Kernel

To search and retrieve documents, we will use the `SearchAsync` method of Semantic memory that encapsulates retrieval abstractions of different memory providers. This method takes in a query and returns a list of documents that are relevant to the query.

In [ ]:
var query = "Ben Graham's investment philosophy";
Console.WriteLine(query + "\n");

var results = memory.SearchAsync(collection: memoryCollectionName, query, limit: 2);
await foreach(var result in results)
{
    Console.WriteLine("   " + result.Metadata.Text);
    Console.WriteLine("   Relevance: " + result.Relevance + "\n");
}

## Grounding Miyagi prompts with SK's Memory "recall"

> Note that this prompt template (semantic function) is located under services/reccommendation-service/dotnet/plugins/AdvisorPlugin

In [ ]:
using Microsoft.SemanticKernel.Plugins.Memory;

// recall is from the TextMemoryPlugin, which has a recall method that can be invoked from Semantic Function to perform retrieval
kernel.ImportFunctions(new TextMemoryPlugin(memory));

var pluginFolder = $"{recommendationServicePath}/plugins";
var advisorPlugin = kernel.ImportSemanticFunctionsFromDirectory(pluginFolder, "AdvisorPlugin");
advisorPlugin

### Set context variables

In [ ]:
using System.Text.Json;

var context = kernel.CreateNewContext();

// Set the parameters for the TextMemorySkill
context.Variables[TextMemoryPlugin.CollectionParam] = memoryCollectionName;
context.Variables[TextMemoryPlugin.RelevanceParam] = "0.8";
context.Variables[TextMemoryPlugin.LimitParam] = "3";

// Set the parameters for the AdvisorPlugin
var stocks = new[] {
    new {symbol = "MSFT", allocation = 0.3},
    new {symbol = "ACN", allocation = 0.1},
    new {symbol = "JPM", allocation = 0.3},
    new {symbol = "PEP", allocation = 0.3}
};
context.Variables["stocks"] = JsonSerializer.Serialize(stocks);

context.Variables["userId"] = "50";
context.Variables["voice"] = "Jim Cramer";
context.Variables["risk"] = "aggressive";

context

### Create native function

In [ ]:
using System.ComponentModel;

/// <summary>
///     UserProfilePlugin shows a native skill example to look user info given userId.
/// </summary>
/// <example>
///     Usage: kernel.ImportSkill("UserProfilePlugin", new UserProfilePlugin());
///     Examples:
///     SKContext["userId"] = "000"
///     {{UserProfilePlugin.GetUserAge $userId }} => {userProfile}
/// </example>
public class UserProfilePlugin
{
    /// <summary>
    ///     Name of the context variable used for UserId.
    /// </summary>
    public const string UserId = "UserId";

    private const string DefaultUserId = "40";
    private const int DefaultAnnualHouseholdIncome = 150000;
    private const int Normalize = 81;

    /// <summary>
    ///     Lookup User's age for a given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetUserAge")]
    [Description("Given a userId, get user age")]
    public string GetUserAge(
        [Description("Unique identifier of a user")]
        string userId)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        int age;

        if (int.TryParse(userId, out var parsedUserId))
            age = parsedUserId > 100 ? parsedUserId % Normalize : parsedUserId;
        else
            age = int.Parse(DefaultUserId);

        // invoke a service to get the age of the user, given the userId
        return age.ToString();
    }

    /// <summary>
    ///     Lookup User's annual income given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetAnnualHouseholdIncome")]
    [Description("Given a userId, get user annual household income")]
    public string GetAnnualHouseholdIncome(
        [Description("Unique identifier of a user")]
        string userId)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        var random = new Random();
        var randomMultiplier = random.Next(1000, 8000);

        // invoke a service to get the annual household income of the user, given the userId
        var annualHouseholdIncome = int.TryParse(userId, out var parsedUserId)
            ? parsedUserId * randomMultiplier
            : DefaultAnnualHouseholdIncome;

        return annualHouseholdIncome.ToString();
    }
}

In [ ]:
// import the UserProfilePlugin
kernel.ImportFunctions(new UserProfilePlugin(), "UserProfilePlugin");

### Invoke the LLM

In [ ]:
using Microsoft.SemanticKernel.TemplateEngine.Basic;

var promptRenderer = new BasicPromptTemplateEngine();
var renderedPrompt = await promptRenderer.RenderAsync(advisorPlugin["InvestmentAdvise"].ToString(), context);
Console.WriteLine(renderedPrompt);

In [ ]:
context

In [ ]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var textToSummarize = @"
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
";

var tldrFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.2, TopP = 0.5 });

var summaryResult = await kernel.RunAsync(textToSummarize, tldrFunction);
var summary = summaryResult.GetValue<string>();

Console.WriteLine(summary);

In [ ]:
var result = await advisorPlugin["PortfolioAllocation"].InvokeAsync(context);

// var result = await kernel.RunAsync(context.Variables, advisorPlugin["PortfolioAllocation"]);

result.GetValue<string>()

![RaG Workflow](../../../../assets/images/sk-memory-orchestration.png)